In [294]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from os.path import dirname, join as pjoin
from scipy import io as sio
#from keras.utils.np_utils import to_categorical

In [295]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import datasets
from tensorflow.keras import optimizers

##### This is where the issue is!
##### The "num_classes" must be 26. NOT 37.
##### Although 26 gives an error I cant make sense of.

In [289]:
#num_classes = 26    THIS IS THE ISSUE!!!!
num_classes = 37
batch_size = 1024
epochs = 50
neurons = 2048

mat_contents = sio.loadmat('ANN-Handwritten-Char-Recognition-master/matlab/emnist-letters.mat')
data = mat_contents['dataset']

X_train = data['train'][0,0]['images'][0,0]
y_train = data['train'][0,0]['labels'][0,0]
X_test = data['test'][0,0]['images'][0,0]
y_test = data['test'][0,0]['labels'][0,0]

val_start = X_train.shape[0] - X_test.shape[0]
X_val = X_train[val_start:X_train.shape[0],:]
y_val = y_train[val_start:X_train.shape[0]]
X_train = X_train[0:val_start,:]
y_train = y_train[0:val_start]

#X_train = X_train.reshape( (X_train.shape[0], 28, 28), order='F')

#convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], "training samples")
print(X_val.shape[0], "validation samples")
print(X_test.shape[0], "test samples")
print(X_val.shape, y_val.shape)

104000 training samples
20800 validation samples
20800 test samples
(20800, 784) (20800, 37)


In [290]:
model = tf.keras.Sequential()
model.add(layers.Dense(neurons, activation='relu', input_shape=(784,)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(neurons, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_123 (Dense)            (None, 2048)              1607680   
_________________________________________________________________
dropout_82 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_124 (Dense)            (None, 2048)              4196352   
_________________________________________________________________
dropout_83 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_125 (Dense)            (None, 37)                75813     
Total params: 5,879,845
Trainable params: 5,879,845
Non-trainable params: 0
_________________________________________________________________


In [291]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_data = (X_val, y_val))

Train on 104000 samples, validate on 20800 samples
Epoch 1/50
104000/104000 [==============================] - 22s 212us/sample - loss: 26.9457 - accuracy: 0.5435 - val_loss: 2.1571 - val_accuracy: 0.6318
Epoch 2/50
104000/104000 [==============================] - 21s 203us/sample - loss: 1.3189 - accuracy: 0.7114 - val_loss: 0.7219 - val_accuracy: 0.7907
Epoch 3/50
104000/104000 [==============================] - 21s 203us/sample - loss: 0.7234 - accuracy: 0.7993 - val_loss: 0.5465 - val_accuracy: 0.8405
Epoch 4/50
104000/104000 [==============================] - 21s 207us/sample - loss: 0.5978 - accuracy: 0.8309 - val_loss: 0.5252 - val_accuracy: 0.8413
Epoch 5/50
104000/104000 [==============================] - 23s 224us/sample - loss: 0.5333 - accuracy: 0.8482 - val_loss: 0.5155 - val_accuracy: 0.8563
Epoch 6/50
104000/104000 [==============================] - 25s 238us/sample - loss: 0.4933 - accuracy: 0.8583 - val_loss: 0.4452 - val_accuracy: 0.8695
Epoch 7/50
104000/104000 [====

In [292]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.9450591341834786
Test accuracy: 0.900625


### num_classes = 28
### first Layer = 512, second layer = 512
#####             5  epochs, 128  batch size - 61.13%
#####             5  epochs,  64  batch size - 60.20%
#####             5  epochs, 128  batch size - 80.91%
#####             10 epochs, 128  batch size - 66.65%
#####             15 epochs, 128  batch size - 63.86%
#####             7  epochs, 128  batch size - 78.78%
#####             7  epochs, 256  batch size - 83.56%
#####             15 epochs, 256  batch size - 82.96%
#####             10 epochs, 256  batch size - 84.33%
#####             15 epochs, 512  batch size - 86.56%
#####             20 epochs, 512  batch size - 87.17%
#####             20 epochs, 1024 batch size - 88.74%
#####             50 epochs, 1024 batch size - 89.59%
#####             50 epochs, 2048 batch size - 89.97%
#####             100epochs, 4098 batch size - 90.44%
            
### num_classes = 37
### first Layer = 512, second layer = 512
#####             50 epochs, 4098 batch size - 90.28%
#####             50 epochs, 2048 batch size - 90.48%
#####             100epochs, 2048 batch size - 90.50%
#####             100epochs, 1024 batch size - 89.50%                        

### first Layer = 1024, second layer = 1024
#####             50 epochs, 1024 batch size - 89.14%
#####             50 epochs, 2048 batch size - 91.14%
#####             50 epochs, 4096 batch size - 90.39%
            
### first Layer = 2048, second layer = 2048
#####             50 epochs, 1024 batch size - 90.06%
#####             50 epochs, 2048 batch size - 90.24%
#####             50 epochs, 4096 batch size - 90.78%
            
            